## Phase 1: Setup and Inspection of Raw Data

### Step 1: Import Required Libraries
Import all necessary packages for data processing, logging, and file handling.

In [1]:
from pathlib import Path          # file/folder paths (works on Windows/macOS)
import pandas as pd               # data loading + cleaning
import logging                    # run log (what happened + when)
import re                         # pattern matching and string manipulation
import unicodedata                # Unicode character properties and normalization
import sys                        # system-specific parameters and functions
import glob                       # file path pattern matching and expansion
import numpy as np                # numerical computing and array operations

### Step 2: Setup Project Folder Structure
Configure paths and create all required directories for the pipeline.

In [2]:
# ===============================
# Project root (K & K Hair Emporium folder)
# ===============================
ROOT = Path.cwd()

# ===============================
# Logs setup (MUST come first)
# ===============================
LOGS_DIR = ROOT / "K & K Hair Emporium_logs"
LOGS_DIR.mkdir(parents=True, exist_ok=True)

LOG_FILE = LOGS_DIR / "setup.log"

logging.basicConfig(
    filename=LOG_FILE,
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

logging.info("Starting folder setup process.")

# ===============================
# Dataset base folder
# ===============================
DATASET_DIR = ROOT / "K & K Hair Emporium_dataset"
K_AND_K_RAW_DIR = DATASET_DIR / "k & k hair emporium_raw"

# ===============================
# Subfolders
# ===============================
RAW_DIR = K_AND_K_RAW_DIR / "k & k hair emporium_raw"
PROCESSED_DIR = K_AND_K_RAW_DIR / "k & k hair emporium_processed"
GARBAGE_DIR = K_AND_K_RAW_DIR / "k & k hair emporium_garbage"
INGESTED_DIR = K_AND_K_RAW_DIR / "k & k hair emporium_ingested"

# ===============================
# Folder creation function
# ===============================
def create_folders():
    folders = [
        RAW_DIR,
        PROCESSED_DIR,
        GARBAGE_DIR,
        INGESTED_DIR,
        LOGS_DIR,
    ]

    for folder in folders:
        folder.mkdir(parents=True, exist_ok=True)
        logging.info(f"Ensured folder exists: {folder}")

# ===============================
# Create the folders
# ===============================
create_folders()

### Step 3: Inspect the Raw K & K Hair Emporium CSV
Load and preview the unprocessed raw data file for initial inspection.

In [3]:
# Load the raw CSV for inspection
CSV_FILE = RAW_DIR / "K & K  Emporium_raw.csv"

if CSV_FILE.exists():
    print(f"✓ CSV file found: {CSV_FILE}")
    print(f"  File size: {CSV_FILE.stat().st_size:,} bytes")
else:
    print(f"✗ ERROR: CSV file not found at {CSV_FILE}")
    raise FileNotFoundError(f"Missing file: {CSV_FILE}")

# Load data
df_raw = pd.read_csv(CSV_FILE)

# Display basic information
print(f"\nShape: {df_raw.shape[0]} rows × {df_raw.shape[1]} columns")
print("\nColumns:")
print(df_raw.columns.tolist())
print("\nFirst 5 rows:")
print(df_raw.head())

✓ CSV file found: \\bca-org-00\Users Folder\jalleyne\Desktop\Protexxa\data_cleaning_scripts\K & K Hair Emporium_dataset\K & K Hair Emporium_dataset\k & k hair emporium_raw\k & k hair emporium_raw\K & K  Emporium_raw.csv
  File size: 188,373 bytes

Shape: 1000 rows × 15 columns

Columns:
['  Name  nsuef', 'Email Addresshbjjb ', ' Password 12582', 'Salt %^bh ', 'Date of Birth   ', 'Street Name  9x9x ', 'Parish   hbjjb', ' Zip   12582', 'Country  %^bh', 'Account Branch   ', 'Loyalty Pointsnsuef ', 'Gender  ', 'Contact Number  (raw) ', 'Signup Date 2018 ', 'Total Purchases  ']

First 5 rows:
        Name  nsuef                    Email Addresshbjjb     Password 12582  \
0  Shanice O'Connor     Shanice.OConnor@KKHAIREMPORIUM.COM            BD#dz*   
1       Renée Lewis               RENE.LEWIS@kandk-hair.co  CPhDeOZIiBOB@Y6s   
2    Rohan Williams      Rohan.Williams@kkhairemporium.com            UNEhEk   
3    Devon Baptiste  Devon.Baptiste.442@loyalty.kkhair.com            mhZRnF   
4    

In [4]:
# ===============================
# Display Columns with Indexes
# ===============================

print("\nCOLUMNS BY INDEX:")
print("-" * 80)
for idx, col in enumerate(df_raw.columns):
    print(f"[{idx}] {col}")
print("-" * 80)


COLUMNS BY INDEX:
--------------------------------------------------------------------------------
[0]   Name  nsuef
[1] Email Addresshbjjb 
[2]  Password 12582
[3] Salt %^bh 
[4] Date of Birth   
[5] Street Name  9x9x 
[6] Parish   hbjjb
[7]  Zip   12582
[8] Country  %^bh
[9] Account Branch   
[10] Loyalty Pointsnsuef 
[11] Gender  
[12] Contact Number  (raw) 
[13] Signup Date 2018 
[14] Total Purchases  
--------------------------------------------------------------------------------


## Phase 2: Data Cleaning and Column Selection

### Step 5: Drop Unnecessary Columns
Keep only the essential columns (indexes 0, 1, 3, 4) and move dropped columns to garbage folder.


In [5]:
# ===============================
# Define columns to keep and drop
# ===============================
KEEP_INDEXES = [0, 1, 3, 4]
ALL_INDEXES = list(range(len(df_raw.columns)))
DROP_INDEXES = [idx for idx in ALL_INDEXES if idx not in KEEP_INDEXES]

# Get column names
KEEP_COLUMNS = [df_raw.columns[idx] for idx in KEEP_INDEXES]
DROP_COLUMNS = [df_raw.columns[idx] for idx in DROP_INDEXES]

print("=" * 80)
print("COLUMN SELECTION")
print("=" * 80)
print(f"\nKEEPING {len(KEEP_COLUMNS)} columns:")
for idx, col in zip(KEEP_INDEXES, KEEP_COLUMNS):
    print(f"  [{idx}] {col}")

print(f"\nDROPPING {len(DROP_COLUMNS)} columns to GARBAGE:")
for idx, col in zip(DROP_INDEXES, DROP_COLUMNS):
    print(f"  [{idx}] {col}")

# ===============================
# Create cleaned and garbage dataframes
# ===============================
df_cleaned = df_raw[KEEP_COLUMNS].copy()
df_garbage = df_raw[DROP_COLUMNS].copy()

# Add index column to garbage dataframe for reference
df_garbage.insert(0, 'original_index', range(len(df_garbage)))

# ===============================
# Save garbage columns
# ===============================
GARBAGE_FILE = GARBAGE_DIR / "dropped_columns.csv"
df_garbage.to_csv(GARBAGE_FILE, index=False)
print(f"\n✓ Dropped columns saved to: {GARBAGE_FILE}")

# ===============================
# Save cleaned data
# ===============================
INGESTED_FILE = INGESTED_DIR / "df_cleaned.csv"
df_cleaned.to_csv(INGESTED_FILE, index=False)
print(f"✓ Cleaned data saved to: {INGESTED_FILE}")

# Log the operation
logging.info(f"Dropped {len(DROP_COLUMNS)} columns and kept {len(KEEP_COLUMNS)} columns")
logging.info(f"Garbage columns saved to {GARBAGE_FILE}")
logging.info(f"Cleaned data saved to {INGESTED_FILE}")

print("\n" + "=" * 80)
print(f"CLEANED DATA SHAPE: {df_cleaned.shape[0]} rows × {df_cleaned.shape[1]} columns")
print("=" * 80)
print("\nCleaned data preview:")
print(df_cleaned.head())

COLUMN SELECTION

KEEPING 4 columns:
  [0]   Name  nsuef
  [1] Email Addresshbjjb 
  [3] Salt %^bh 
  [4] Date of Birth   

DROPPING 11 columns to GARBAGE:
  [2]  Password 12582
  [5] Street Name  9x9x 
  [6] Parish   hbjjb
  [7]  Zip   12582
  [8] Country  %^bh
  [9] Account Branch   
  [10] Loyalty Pointsnsuef 
  [11] Gender  
  [12] Contact Number  (raw) 
  [13] Signup Date 2018 
  [14] Total Purchases  

✓ Dropped columns saved to: \\bca-org-00\Users Folder\jalleyne\Desktop\Protexxa\data_cleaning_scripts\K & K Hair Emporium_dataset\K & K Hair Emporium_dataset\k & k hair emporium_raw\k & k hair emporium_garbage\dropped_columns.csv
✓ Cleaned data saved to: \\bca-org-00\Users Folder\jalleyne\Desktop\Protexxa\data_cleaning_scripts\K & K Hair Emporium_dataset\K & K Hair Emporium_dataset\k & k hair emporium_raw\k & k hair emporium_ingested\df_cleaned.csv

CLEANED DATA SHAPE: 1000 rows × 4 columns

Cleaned data preview:
        Name  nsuef                    Email Addresshbjjb         Sal

# Ensure column names match expected list

In [6]:

EXPECTED_COLS = ["Name", "Email Address", "Salt", "Date of Birth"]
old_columns = df_cleaned.columns.tolist()
if len(old_columns) == len(EXPECTED_COLS):
    # save old names to garbage file
    oldname_file = GARBAGE_DIR / "old_column_names.csv"
    pd.DataFrame({'old_name': old_columns}).to_csv(oldname_file, index=False)
    print(f"✓ Old column names written to {oldname_file}")

    df_cleaned.columns = EXPECTED_COLS
    print("Renamed cleaned dataframe columns to expected names:")
    print(df_cleaned.columns.tolist())
else:
    print(f"⚠️  Column count ({len(old_columns)}) does not match expected ({len(EXPECTED_COLS)}); skipping rename")
    print("Current columns:", old_columns)

# Log column renaming
logging.info(f"Old column names saved: {oldname_file if 'oldname_file' in locals() else 'none'}")
logging.info(f"Final column names: {df_cleaned.columns.tolist()}")


✓ Old column names written to \\bca-org-00\Users Folder\jalleyne\Desktop\Protexxa\data_cleaning_scripts\K & K Hair Emporium_dataset\K & K Hair Emporium_dataset\k & k hair emporium_raw\k & k hair emporium_garbage\old_column_names.csv
Renamed cleaned dataframe columns to expected names:
['Name', 'Email Address', 'Salt', 'Date of Birth']


### Step 6: Validate Kept Columns for Whitespace
Inspect the cleaned dataset to ensure no leading/trailing spaces remain in the retained columns.

In [7]:
# Check for leading/trailing whitespace in cleaned data columns
print("\nWHITESPACE CHECK ON CLEANED DATA:")
problem_cols = {}
for col in df_cleaned.columns:  # iterate actual dataframe columns
    if df_cleaned[col].dtype == 'object':
        count = df_cleaned[col].astype(str).str.match(r"^\s|\s$").sum()
        if count > 0:
            problem_cols[col] = count

if problem_cols:
    print("Columns with whitespace issues:")
    for col, cnt in problem_cols.items():
        print(f" - {col}: {cnt} cells")
    # show a snippet of up to 10 rows
    print("\nSample rows with issues (up to 10):")
    mask = False
    for col in problem_cols:
        mask |= df_cleaned[col].astype(str).str.match(r"^\s|\s$")
    display(df_cleaned[mask].head(10))
else:
    print("✓ No leading/trailing spaces detected in cleaned columns.")


WHITESPACE CHECK ON CLEANED DATA:
Columns with whitespace issues:
 - Name: 89 cells

Sample rows with issues (up to 10):


,Name,Email Address,Salt,Date of Birth
6,Tia Brathwaite,Tia.Brathwaite@mail.kkhair.net,3c681d06bd2aa399,1984/04/06
35,Sofía Greenidge,Sofa.Greenidge-387@@kkhairemporium.com,b98bfe3fa6bad1__,2006/12/16
44,Khadija De la Cruz,Khadija.Cruz-711@kandk-hair.co,44205eb64de62343,1989/06/14
47,Jéssica Jean-Baptiste,Jssica.Jean-Baptiste@@loyalty.kkhair.com,c9367df148217dbe,1962/08/14
64,Kirsten Brathwaite St. Clair,Kirsten.Clair.600@KKHAIREMPORIUM.COM,49b04310c296b6d4,1974/10/22
69,Latoya Núñez,Latoya.Nez@#.kkhairemporium.com,751b70528cbcc602,2004/4/22
74,Anaïs Alleyne,Anas.Alleyne@loyalty.kkhair.com,f6a041053984e072,1999/04
89,José Núñez,Jos.Nez@mail.kkhair.net,7dcacccd65f46cbd,1966-04-06
95,Jermaine Campbell,Jermaine.Campbell@MAIL.KKHAIR.NET,d7515f29bd07633b,2003/06/03
97,Latoya García,Latoya.Garca@kandk-hair.co,0d64b9720f95e0ee,1991/05


# Duplicate row analysis on cleaned data

In [8]:

duplicates = df_cleaned.duplicated(keep=False)
num_dup = duplicates.sum()
print(f"\nDUPLICATE ROWS: {num_dup} total ({(num_dup/len(df_cleaned)*100):.2f}% of dataset)")

dup_file = None
if num_dup > 0:
    dup_rows = df_cleaned[duplicates]
    print("\nSample duplicates:")
    display(dup_rows.head(10))
    # save duplicates to processed folder
    dup_file = PROCESSED_DIR / "duplicate_rows.csv"
    dup_rows.to_csv(dup_file, index=False)
    print(f"✓ Exported duplicates to {dup_file}")

    # also move duplicates to garbage and remove them from df_cleaned
    garbage_dup_file = GARBAGE_DIR / "duplicate_rows.csv"
    dup_rows.to_csv(garbage_dup_file, index=False)
    df_cleaned = df_cleaned.drop(dup_rows.index)
    print(f"✓ Dropped duplicate rows from cleaned data and wrote to {garbage_dup_file}")
else:
    print("✓ No duplicate rows found in cleaned data.")

# check for duplicate values within each column
print("\nDUPLICATE VALUES WITHIN COLUMNS:")
for col in df_cleaned.columns:
    dup_vals = df_cleaned[col][df_cleaned[col].duplicated()]
    if not dup_vals.empty:
        print(f" - {col}: {len(dup_vals)} duplicate entries")

print("\nDuplicate check complete.")


DUPLICATE ROWS: 60 total (6.00% of dataset)

Sample duplicates:


,Name,Email Address,Salt,Date of Birth
5,Khadija Baptiste,KHADIJA.BAPTISTE@loyalty.kkhair.com,b4d419b1b673bd47,1970/11/19
190,Renée Holder,RENE.HOLDER@loyalty.kkhair.com,c752344fd8be7e12,1958-06-08
206,Tyrone Forde,Tyrone.Forde@kkhairemporium.com,2b7fdd13888ad8__,1998/07
218,Latoya Brathwaite,Latoya.Brathwaite@mail.kkhair.net,cc18cabdda4b86f9,1995/05/12
243,Anaïs Hernández,Anas.Hernndez@loyalty.kkhair.com,7f0348598d5d65b3,1980/11/27
263,Keisha Ramdeen,Keisha.Ramdeen@kkhairemporium.com,7d8949a229bee1e2,1971/01
285,Anaïs Charles,Anas.Charles.635@kandk-hair.co,8cd419af4f38e48c,1963/01/13
465,Kareem Alleyne,Kareem.Alleyne@kandk-hair.co,4f98f1c6d2a4fca9,1968-12-06
503,Noël Best,Nol.Best@kkhairemporium.com,a57a3d675d8f2112,1996/09/23
576,Sade Joseph,Sade.Joseph@loyalty.kkhair.com,9cbd92a9d6a3eaf6,1961/02/05


✓ Exported duplicates to \\bca-org-00\Users Folder\jalleyne\Desktop\Protexxa\data_cleaning_scripts\K & K Hair Emporium_dataset\K & K Hair Emporium_dataset\k & k hair emporium_raw\k & k hair emporium_processed\duplicate_rows.csv
✓ Dropped duplicate rows from cleaned data and wrote to \\bca-org-00\Users Folder\jalleyne\Desktop\Protexxa\data_cleaning_scripts\K & K Hair Emporium_dataset\K & K Hair Emporium_dataset\k & k hair emporium_raw\k & k hair emporium_garbage\duplicate_rows.csv

DUPLICATE VALUES WITHIN COLUMNS:
 - Name: 238 duplicate entries
 - Email Address: 26 duplicate entries
 - Salt: 39 duplicate entries
 - Date of Birth: 53 duplicate entries

Duplicate check complete.


# Standardize Date of Birth format

In [9]:
if "Date of Birth" in df_cleaned.columns:
    # parse as datetime if not already
    df_cleaned["Date of Birth"] = pd.to_datetime(df_cleaned["Date of Birth"], errors='coerce')
    # format YYYY/MM/DD
    df_cleaned["Date of Birth"] = df_cleaned["Date of Birth"].dt.strftime("%Y/%m/%d")
    print("Date of Birth column standardized to YYYY/MM/DD format. Sample:")
    print(df_cleaned["Date of Birth"].head())
else:
    print("Date of Birth column not found in dataframe.")


Date of Birth column standardized to YYYY/MM/DD format. Sample:
0    1958/11/24
1    1996/12/28
2           NaN
3    1974/03/06
4    2000/05/20
Name: Date of Birth, dtype: object


# Remove rows with invalid/missing date of birth

In [10]:
if "Date of Birth" in df_cleaned.columns:
    # identify invalid rows: NaN, empty, or not matching YYYY/MM/DD format
    invalid_mask = (df_cleaned["Date of Birth"].isna()) | \
                   (df_cleaned["Date of Birth"].astype(str).str.strip() == '') | \
                   (~df_cleaned["Date of Birth"].astype(str).str.match(r'^\d{4}/\d{2}/\d{2}$'))
    
    num_invalid = invalid_mask.sum()
    print(f"\nINVALID DATE OF BIRTH RECORDS: {num_invalid} rows")
    
    if num_invalid > 0:
        invalid_rows = df_cleaned[invalid_mask]
        print("\nSample invalid records:")
        display(invalid_rows.head(10))
        
        # save to garbage folder
        invalid_dob_file = GARBAGE_DIR / "invalid_dates.csv"
        invalid_rows.to_csv(invalid_dob_file, index=False)
        print(f"✓ Invalid records saved to {invalid_dob_file}")
        
        # remove from df_cleaned
        df_cleaned = df_cleaned[~invalid_mask]
        print(f"✓ Removed {num_invalid} rows with invalid/missing dates")
        print(f"  Cleaned data now has {len(df_cleaned)} rows")
        
        logging.info(f"Removed {num_invalid} rows with invalid/missing Date of Birth")
        logging.info(f"Cleaned data rows after removal: {len(df_cleaned)}")
    else:
        print("✓ All Date of Birth records are valid.")
else:
    print("Date of Birth column not found.")



INVALID DATE OF BIRTH RECORDS: 314 rows

Sample invalid records:


,Name,Email Address,Salt,Date of Birth
2,Rohan Williams,Rohan.Williams@kkhairemporium.com,f244f58d669cbee3,NaN
8,Tyrone Campbell,Tyrone.Campbell@loyalty.kkhair.com,d7f78df0cac5e40c,NaN
9,Kareem Campbell,KAREEM.CAMPBELL@KANDK-HAIR.CO,359309cc6273931b,NaN
13,José Baptiste,JOS.BAPTISTE@mail.kkhair.net,bfe16849ef307590,NaN
14,Sade Núñez,Sade.Nez@mail.kkhair.net,fcc4f14a3e3e04d4,NaN
19,Noël De la Cruz,Nol.Cruz-615@loyalty.kkhair.com,d080589ab054c240,NaN
20,Dwayne De la Cruz,DWAYNE.CRUZ-614@kkhairemporium.com,c7b2c1d0e2adcd93,NaN
22,Marlon St. Clair,Marlon.Clair-940@loyalty.kkhair.com,29ca42db0b956af6,NaN
23,Marlon Charles,Marlon.Charles@kandk-hair.co,NaN,NaN
27,Khadija Best,Khadija.Bestkkhairemporium.com,a7b92868492545a1,NaN


✓ Invalid records saved to \\bca-org-00\Users Folder\jalleyne\Desktop\Protexxa\data_cleaning_scripts\K & K Hair Emporium_dataset\K & K Hair Emporium_dataset\k & k hair emporium_raw\k & k hair emporium_garbage\invalid_dates.csv
✓ Removed 314 rows with invalid/missing dates
  Cleaned data now has 626 rows


# Save final cleaned data with correct column names

# ensure column names still correct before final save

In [11]:
# Validate email addresses if present
if "Email Address" in df_cleaned.columns:
    # simple regex for basic email validation
    email_pattern = r'^[^@\s]+@[^@\s]+\.[^@\s]+$'
    invalid_email_mask = ~df_cleaned["Email Address"].astype(str).str.match(email_pattern)

    num_invalid_emails = invalid_email_mask.sum()
    print(f"\nINVALID EMAIL RECORDS: {num_invalid_emails} rows")

    if num_invalid_emails > 0:
        invalid_email_rows = df_cleaned[invalid_email_mask]
        print("\nSample rows with invalid email formats:")
        display(invalid_email_rows.head(10))

        # save invalid emails to garbage folder
        email_garbage_file = GARBAGE_DIR / "invalid_emails.csv"
        invalid_email_rows.to_csv(email_garbage_file, index=False)
        print(f"✓ Invalid email rows saved to {email_garbage_file}")

        # remove them from the cleaned dataframe
        df_cleaned = df_cleaned[~invalid_email_mask]
        print(f"✓ Removed {num_invalid_emails} rows with invalid emails")
    else:
        print("✓ All email addresses appear valid.")
else:
    print("Email Address column not found in dataframe.")



INVALID EMAIL RECORDS: 53 rows

Sample rows with invalid email formats:


,Name,Email Address,Salt,Date of Birth
29,Tanesha García,Tanesha.Garca@@mail.kkhair.net,a5c9af9f0ba3d90f,2006/11/24
34,Kofi Morris,Kofi.Morris@ kkhairemporium.com,47d9815df1bcadd4,1971/07/01
35,Sofía Greenidge,Sofa.Greenidge-387@@kkhairemporium.com,b98bfe3fa6bad1__,2006/12/16
47,Jéssica Jean-Baptiste,Jssica.Jean-Baptiste@@loyalty.kkhair.com,c9367df148217dbe,1962/08/14
56,Sofía De la Cruz,Sofa.Cruz@,9e3f12f63c9d1446,2006/01/31
65,Mikaël Thompson,Mikal.Thompson@,afeeb9f352846822,2000/07/02
66,Noël Pérez,Nol.Prez_184@,bcfe11a3885ccb28,1974/12/30
82,Rohan Campbell-Pérez,Rohan.Campbell-Prez@,13ee17c1c169ec99,1992/11/25
90,Tanesha Best,TANESHA.BEST-722loyalty.kkhair.com,27f291a0fefadb99,1982/04/12
99,Anaïs Joseph,Anas.Josephloyalty.kkhair.com,595545731a4e8b56,1958/02/10


✓ Invalid email rows saved to \\bca-org-00\Users Folder\jalleyne\Desktop\Protexxa\data_cleaning_scripts\K & K Hair Emporium_dataset\K & K Hair Emporium_dataset\k & k hair emporium_raw\k & k hair emporium_garbage\invalid_emails.csv
✓ Removed 53 rows with invalid emails


In [12]:
EXPECTED_COLS = ["Name", "Email Address", "Salt", "Date of Birth"]
if list(df_cleaned.columns) != EXPECTED_COLS and len(df_cleaned.columns) == len(EXPECTED_COLS):
    df_cleaned.columns = EXPECTED_COLS
    print("Reapplied expected column names before final save.")

print("\nFINAL CLEANED DATA SUMMARY:")
print(f"Shape: {df_cleaned.shape[0]} rows × {df_cleaned.shape[1]} columns")
print(f"Columns: {df_cleaned.columns.tolist()}")
print("\nFirst 5 rows of final cleaned data:")
display(df_cleaned.head())

# Save the final cleaned data with proper column names
final_cleaned_file = INGESTED_DIR / "df_cleaned_final.csv"
df_cleaned.to_csv(final_cleaned_file, index=False)
print(f"\n✓ Final cleaned data saved to {final_cleaned_file}")

logging.info(f"Final cleaned data shape: {df_cleaned.shape}")
logging.info(f"Final column names: {df_cleaned.columns.tolist()}")
logging.info(f"Final cleaned data saved to {final_cleaned_file}")


FINAL CLEANED DATA SUMMARY:
Shape: 573 rows × 4 columns
Columns: ['Name', 'Email Address', 'Salt', 'Date of Birth']

First 5 rows of final cleaned data:


,Name,Email Address,Salt,Date of Birth
0,Shanice O'Connor,Shanice.OConnor@KKHAIREMPORIUM.COM,e805da846a32c3bb,1958/11/24
1,Renée Lewis,RENE.LEWIS@kandk-hair.co,78dcb74f21345d2c,1996/12/28
3,Devon Baptiste,Devon.Baptiste.442@loyalty.kkhair.com,e4d58e72e310275d,1974/03/06
4,Devon Baptiste,Devon.Baptiste@KKHAIREMPORIUM.COM,a86a78c49ea20e32,2000/05/20
6,Tia Brathwaite,Tia.Brathwaite@mail.kkhair.net,3c681d06bd2aa399,1984/04/06



✓ Final cleaned data saved to \\bca-org-00\Users Folder\jalleyne\Desktop\Protexxa\data_cleaning_scripts\K & K Hair Emporium_dataset\K & K Hair Emporium_dataset\k & k hair emporium_raw\k & k hair emporium_ingested\df_cleaned_final.csv


# verify total row counts to ensure no data lost and everything accounted for

# gather counts by inspecting saved garbage/processed CSV files on disk

# only count each removal category once (processed duplicate file is authoritative)

In [13]:
print("\n=== DATAQUANTITY CHECK ===")
print(f"Original raw data rows: {len(df_raw)}")
print(f"Final cleaned data rows: {len(df_cleaned)}")
removed = len(df_raw) - len(df_cleaned)
print(f"Rows removed during cleaning: {removed}")

data_loss = removed > 0
print(f"Data loss occurred? {data_loss}")


file_counts = {}

paths = {
    'duplicate_rows': PROCESSED_DIR / 'duplicate_rows.csv',
    'invalid_dates': GARBAGE_DIR / 'invalid_dates.csv',
    'invalid_emails': GARBAGE_DIR / 'invalid_emails.csv',
}
for label, path in paths.items():
    if path.exists():
        try:
            df_tmp = pd.read_csv(path)
            file_counts[label] = len(df_tmp)
        except Exception:
            file_counts[label] = None

if file_counts:
    print("\nCounts from exported files:")
    for k, v in file_counts.items():
        print(f" - {k}: {v if v is not None else 'error reading'} rows")
    # sum valid numeric counts
    numeric = [v for v in file_counts.values() if isinstance(v, (int, float))]
    total_exported = sum(numeric)
    print(f"Sum of exported rows: {total_exported}")
    accounted = (len(df_cleaned) + total_exported) == len(df_raw)
    print(f"All rows accounted for? {accounted}")
else:
    print("\nNo exported files found for counting.")



=== DATAQUANTITY CHECK ===
Original raw data rows: 1000
Final cleaned data rows: 573
Rows removed during cleaning: 427
Data loss occurred? True

Counts from exported files:
 - duplicate_rows: 60 rows
 - invalid_dates: 314 rows
 - invalid_emails: 53 rows
Sum of exported rows: 427
All rows accounted for? True


# Post-cleaning analysis and output validation

# check existence of key output files

In [14]:
print("\n=== POST-CLEANING ANALYSIS ===")
print(f"Cleaned dataframe shape: {df_cleaned.shape}")
print("Column data types:")
print(df_cleaned.dtypes)
print("\nMissing values by column:")
print(df_cleaned.isna().sum())


def check_file(path):
    exists = path.exists()
    print(f"{path.name}: {'FOUND' if exists else 'MISSING'}")
    return exists

print("\nOutput files status:")
files_to_check = [
    INGESTED_DIR / "df_cleaned.csv",
    INGESTED_DIR / "df_cleaned_final.csv",
    GARBAGE_DIR / "invalid_dates.csv",
    GARBAGE_DIR / "invalid_emails.csv",
    PROCESSED_DIR / "duplicate_rows.csv",
]
for f in files_to_check:
    check_file(f)



=== POST-CLEANING ANALYSIS ===
Cleaned dataframe shape: (573, 4)
Column data types:
Name             object
Email Address    object
Salt             object
Date of Birth    object
dtype: object

Missing values by column:
Name             0
Email Address    0
Salt             0
Date of Birth    0
dtype: int64

Output files status:


df_cleaned.csv: FOUND
df_cleaned_final.csv: FOUND
invalid_dates.csv: FOUND
invalid_emails.csv: FOUND
duplicate_rows.csv: FOUND


# final verification of cleaned data file

In [15]:
print("\n=== FINAL FILE VERIFICATION ===")
final_path = INGESTED_DIR / "df_cleaned_final.csv"
if final_path.exists():
    df_final = pd.read_csv(final_path)
    print(f"Final CSV shape: {df_final.shape}")
    same = df_final.equals(df_cleaned)
    print(f"Loaded file identical to in-memory df_cleaned? {same}")
    print("\nSample from final file:")
    display(df_final.head())
    # extra diagnostics
    print("\nAny missing values?")
    print(df_final.isna().sum())
    print("\nAny duplicates?")
    print(df_final.duplicated().sum())
else:
    print(f"Final file not found at {final_path}")



=== FINAL FILE VERIFICATION ===
Final CSV shape: (573, 4)
Loaded file identical to in-memory df_cleaned? False

Sample from final file:


,Name,Email Address,Salt,Date of Birth
0,Shanice O'Connor,Shanice.OConnor@KKHAIREMPORIUM.COM,e805da846a32c3bb,1958/11/24
1,Renée Lewis,RENE.LEWIS@kandk-hair.co,78dcb74f21345d2c,1996/12/28
2,Devon Baptiste,Devon.Baptiste.442@loyalty.kkhair.com,e4d58e72e310275d,1974/03/06
3,Devon Baptiste,Devon.Baptiste@KKHAIREMPORIUM.COM,a86a78c49ea20e32,2000/05/20
4,Tia Brathwaite,Tia.Brathwaite@mail.kkhair.net,3c681d06bd2aa399,1984/04/06



Any missing values?
Name             0
Email Address    0
Salt             0
Date of Birth    0
dtype: int64

Any duplicates?
0
